In [1]:
# 引入Python包，在jupyter notebook 运行后可以直接输出多个变量 注意大小写
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 指定gpu设备
import os
os.environ['CUDA_VISIBLE_DEVICES'] ="0" # gpu
# os.environ["CUDA_VISIBLE_DEVICES"] = "-1" # cpu

In [2]:
from keras import models
from keras import layers
from keras.models import load_model
from keras import optimizers

model = load_model('cats_and_dogs_small_vgg16_02.h5')

Using TensorFlow backend.



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



D:\ProgramData\Miniconda3\envs\tensorflow_114_gpu\lib\site-packages\keras\engine\saving.py:384: UserWarning: Error in loading the saved optimizer state. As a result, your model is starting with a freshly initialized optimizer.
  warnings.warn('Error in loading the saved optimizer '


In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 4, 4, 512)         14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 8192)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               2097408   
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 257       
Total params: 16,812,353
Trainable params: 16,812,353
Non-trainable params: 0
_________________________________________________________________


In [4]:
print('This is the number of trainable weights ' 
         'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 30


In [8]:
# 冻结vgg16卷积基
for layer in model.layers:
    print(layer, layer.name)
    if layer.name == 'vgg16':
        layer.trainable = False

<keras.engine.training.Model object at 0x00000145CAB66548> vgg16
<keras.layers.core.Flatten object at 0x00000145CAE94988> flatten_1
<keras.layers.core.Dense object at 0x00000145CAE6ABC8> dense_1
<keras.layers.core.Dense object at 0x00000145CAF0CB88> dense_2


In [9]:
print('This is the number of trainable weights ' 
         'before freezing the conv base:', len(model.trainable_weights))

This is the number of trainable weights before freezing the conv base: 4


In [10]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator

base_dir = 'kaggle-dogs-vs-cats/cats_and_dogs_small'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')
test_dir = os.path.join(base_dir, 'test')

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

In [11]:
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array,array_to_img

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255) # 验证数据不能增强

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_dir,
    target_size=(150, 150),
    batch_size=20,
    class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [14]:
# 编译器要在设置 model.trainable = False 之后再配置
model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
                   loss='binary_crossentropy',
                   metrics=['acc'])

history = model.fit_generator(train_generator,
                                    steps_per_epoch=100,
                                    epochs=5,
                                    validation_data=validation_generator,
                                    validation_steps=50)

Epoch 1/5
100/100 [==============================] - 214s 2s/step - loss: 0.2867 - acc: 0.8755 - val_loss: 0.3019 - val_acc: 0.9030
Epoch 2/5
100/100 [==============================] - 414s 4s/step - loss: 0.2745 - acc: 0.8820 - val_loss: 0.3028 - val_acc: 0.8990
Epoch 3/5
100/100 [==============================] - 387s 4s/step - loss: 0.2764 - acc: 0.8815 - val_loss: 0.2692 - val_acc: 0.9000
Epoch 4/5
100/100 [==============================] - 257s 3s/step - loss: 0.2678 - acc: 0.8870 - val_loss: 0.1856 - val_acc: 0.9020
Epoch 5/5
100/100 [==============================] - 177s 2s/step - loss: 0.2711 - acc: 0.8830 - val_loss: 0.6089 - val_acc: 0.8920


In [ ]:
test_generator = test_datagen.flow_from_directory( 
        test_dir, 
        target_size=(150, 150), 
        batch_size=20, 
        class_mode='binary')
model.evaluate_generator(test_generator, steps=50)

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1) 
 
plt.plot(epochs, acc, 'bo', label='Training acc') 
plt.plot(epochs, val_acc, 'b', label='Validation acc') 
plt.title('Training and validation accuracy') 
plt.legend() 
 
plt.figure() 
 
plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validation loss') 
plt.title('Training and validation loss') 
plt.legend()

In [ ]:
model.save('cats_and_dogs_small_vgg16_03.h5')

In [ ]:
max(history.history['val_acc'])